# Table of contents

### Paper questions

### Use case requirements

- [Scaling from testbed to Oilrig](#scaling)

### Analyzed hardware options

- [Point Cloud sensors](#pointCloudSensors)
- [Processor](#processor)
- [Altitude + Syncronism ](#Altitude)
- [Autopilots](#autoPilots)
- [Cameras](#Camera)
- [Wireless dongle](#Wireless)
- [Platform](#Platform)
- [Summary](#Summary)

### [Software requirements](#software)

### [3d Map pipleline](#3dmap)


# Paper questions
1. Where does Lazy Theta* break? Marine environments vs plants (Freiburg)
1. Where do large structures break the naive choice of next frontier point to explore?

2. Comparison of how other heuristics do for that choice

2. Systematic heuristic (top-down; east-west; south-north)

3. Comparison of the impact of update rate of the current goal (waiting until reaching the goal or updating with new information)

4. Impact of integrating the heuristics into the Lazy Theta * search

    1. Data from simulation + changing the resolution of the octree to increase the search space. Be carefull of when it no longer makes sense because of the reach of the sensor (proportion between the number of cells that become explored in each position and the size of the resolution)
    1. Baseline (frontier cells w/ Eucleadian distance) >>>> Compare criteria for selecting frontier cells


# Use case requirements
<a id='scaling'></a>
### Scaling from testbed to Oilrig

- [**Actinia**](http://www.deepwater.com/Documents/RigSpecs/Actinia%20Specs.pdf)  270 ft. long x 255 ft. wide x 116 ft. deep = 82m x 78m x 35m = 223 860m$^3$ => 223 860m / (0.05m$^3$) = 1 791 nodes @ 5cm resolution
- [**Transocean 712**](http://www.deepwater.com/Documents/RigSpecs/Transocean%20712.pdf) 295 ft. long x 249 ft. wide x 130 ft. deep = 90m x 77m x 40m = 277.200m$^3$
- [**Sedco 714**](http://www.deepwater.com/Documents/RigSpecs/Sedco%20714.pdf)  300 ft. long x 267 ft. wide x 130 ft. deep = 92m x 81m x 40m = 298.080m$^3$
- [**Sedco 711**](http://www.deepwater.com/Documents/RigSpecs/Sedco%20711.pdf)  295 ft. long x 249 ft. wide x 130 ft. deep = 90m x 76m x 40m = 273.600m$^3$

# Analyzed hardware options
<a id='pointCloudSensors'></a>
### Point Cloud sensors

##### RGB-D Cameras

- [**Orbbec Astra S**](https://orbbec3d.com/product-astra/) [Even more detailed specifications](https://3dclub.orbbec3d.com/t/advanced-technical-specs-of-persee-and-astra-sensors/56/9) Does not work outdoors during the day. Range 1m is +/-1-3mm
- [**Asus Xtion PRO**](https://www.asus.com/3D-Sensor/Xtion_PRO/specifications/)
- [**Kinect**](https://developer.microsoft.com/en-us/windows/kinect/hardware) Deprecated 20€ Paired up with PCL which is optimized for streaming point clouds, always keeps all the points of the point cloud.

##### Laser
Interesting found from [a list](https://www.dronezon.com/learn-about-drones-quadcopters/best-lidar-sensors-for-drones-great-uses-for-lidar-sensors/).
- [**VLP-16 Lite lidar**](http://www.velodynelidar.com/vlp-16-lite.html) 590 g 100m 
- [**VLP-16 lidar**](http://velodynelidar.com/vlp-16.html): 830g. 100m +- 3cm Too expensive 
- [**VLP-32 lidar**](http://velodynelidar.com/hdl-32e.html): 1300g. Too heavy
- [**LeddarTech M16**](https://autonomoustuff.com/product/leddar-m16-multi-element-sensor-module/) 100m 600€ No rotation. ROS integration? There is one [package for indigo for older products](https://github.com/dispatch-ai/ros-leddar-vu8) Uses a data formart [LaserScan](http://www.ros.org/doc/api/sensor_msgs/html/msg/LaserScan.html)
- [**UTM-30LX-EW**](https://www.hokuyo-aut.jp/search/single.php?serial=223) 4.400€ range 5.6m
- [**UST-10LX**](https://www.hokuyo-aut.jp/search/single.php?serial=167) 1.500€ · 130g · range 10m [comparision of the 3 lasers](https://acroname.com/blog/hokuyo-laser-range-finder-comparison)
- [**URG-04LX-UG01**](https://www.hokuyo-aut.jp/search/single.php?serial=166) 975€ range 30m

<a id='processor'></a>
### Processor

##### RAm & Octree
It needs to have enough RAM for the whole tree. An octree has $ \dfrac{N^{L+1}-1}{N-1}$ nodes. Where N is the branching factor 8 and L is the amount of levels. 

>The actual size of the data structure (76 byte for inner nodes and 12 byte for leaf nodes) is again padded to multiples of the word size (8 byte on a 64-bit architecture) by most compilers.

** 5cm resolution **
The intended volume is much smaller, to cover it (having the drone start at the center of the volume) only 11 tree levels are required: $$8^{11} = 8.589.934.592 > 1.812.720.000 (i.e. Actinia)$$ 
Using the remaining levels as unused aligning margin, the estimate branching factor is of 4 and the worst case scenario of maximum resolution nodes for the whole volume, the total number of nodes in the tree would be:

$$ \dfrac{4^{5+1}-1}{4-1} + \left(\dfrac{8^{15+1}-1}{8-1} - \dfrac{8^{5+1}-1}{8-1} \right) = \dfrac{4^{6}-1}{3} + \dfrac{8^{6}*(8^{10}-1)}{7} = 4,021071092×10^{13} = 40.210.710.920.000$$

The breakdown is 8.5x10$^7$ leaf nodes + 4x10$^{13}$ inner nodes.


** Flight precision is different from data precision!! 50cm resolution**
Horizontal hoovering is 1,5m for Matrice 200 and the safety distance is 3m! So octree resolution can be 0,5 meters. Which will mean 129 Mb for the Actinia volume.
The calculations for volume, node and memory are in [this excel](https://docs.google.com/spreadsheets/d/1UFneS__9-L0viD8dhHrdOTUGh9j_4SgYT0UNm2cHwcY/edit?usp=sharing). 

##### Board and weight
- [**SanCloud BeagleBone Enhanced**](http://beagleboard.org/bone) 1Gb
- NUC 16Gb
- [**Up Board**](http://www.up-board.org/up/specifications/) 1GB/2GB/4GB · +/-150g · [Best option x5-Z8350, 4GB, 80g,	64Gb, 130€](https://up-shop.org/up-boards/43-up-board-4b-32-gb-emmc-memory.html)

<a id='Altitude'></a>
### Altitude + Syncronism 

- [**Here +**](https://www.proficnc.com/content/12-here) RTK CNSS. Compatible with Pixhawk, Pixhawk2 and APM. MPU9250 IMU · 200g · 610€ · cm level accuracy · time pulse syncronization
- [**Spatial**](https://www.advancednavigation.com.au/product/spatial#specifications) RTK option · 3m accuracy, 0.03m with RTK 0.08º (worst) accuracy · PPS · 2974€
- [**Reach**](https://emlid.com/reach/) has [PixHawk integration](https://emlid.com/reach-rtk-naviopixhawk-integration-released/) 14g · 200€ · accuracy?
- [**Drotek XL**]() has [PixHawk integration](https://docs.px4.io/en/peripherals/rtk_gps_drotek_xl.html) 300€ · 23g · 2.5cm accuracy 
- IMU with PPS
- GPS with PPS 
- [Tersus BX305](https://www.tersus-gnss.com/products/bx305kit) 2025€ [48g](https://cdn.shopify.com/s/files/1/0928/6900/files/Precis_BX305_User_Guide.pdf?10976960220533540537)

<a id='autoPilots'></a>
### Auto pilots & specific platforms

- [**Px4 hardware implementations**](http://px4.io/hardware/)
- [**Pixhawk 2**](https://pixhawk.org/modules/pixhawk2) Has some bugs with Px4, ?Fran? has reverted to Pixhawk 1
- [**Pixhawk**](https://pixhawk.org/modules/pixhawk) Better ROS integration
- [**Dropix**](https://drotek.com/en/docs/docs-dropix/)
- APM
- [**N3**](http://www.dji.com/n3/info#specs) 132g · 275€

<a id='Camera'></a>
### Camera + Gimbal

https://filmora.wondershare.com/drones/best-camera-gimbals-to-buy.html
##### Gimbal
- [Gremsy T1](https://store.gremsy.com/product/gremsy-t1/) 700g · 990€
- [Karma Grip](https://shop.gopro.com/EMEA/stabilization) + [Karma Mouting Ring](https://shop.gopro.com/EMEA/accessories-2/karma-mounting-ring/ACOMC-001.html) 242g+100g · +40€
- [Tarot 3axis](https://rc-innovations.es/gimbal-brushless-3-ejes-tarot-gopro-3diii-tl3t01-tarot?language=es&currency=EUR&gclid=Cj0KCQjwm9vPBRCQARIsABAIQYefZgkwsKB5FhJPfQ47wmRjbtorjOyZD7M2IQDQgWB-i0sA9i80IUkaAkIMEALw_wcB) 160g · 145€
- [Tarot 2axis](https://rc-innovations.es/Gimbal-brushless-2-ejes-tarot-gopro?language=es&currency=EUR&gclid=Cj0KCQjwm9vPBRCQARIsABAIQYcGhCggxRoqqbkXW9Yi39pkJUZv7BE8MNPLOIDKyEYHFTceynnI-pUaAp5jEALw_wcB) [3dr store](https://store.3dr.com/products/tarot-t-2d-brushless-gimbal-kit) 195g · 79€

##### Camera
- [Hero 4](https://gopro.com/update/hero4) [86g](https://www.google.es/shopping/product/7654478670727366749?dcr=0&q=hero+4&biw=1920&bih=1103&prds=paur:ClkAsKraX5or_tf2LY7E3KZRQD6vBidhplLsPjTuYye55yUVDSoJxkwxP93-vzo3ShkKoyTyeISinMmldv-i3MEJqQhXKe-HX1HJMrgkm5VB8WCz35uBeS5JkhIZAFPVH7230jn8eIBXjU4ys9_r3dxXWQAJbA&sa=X&ved=0ahUKEwiD5ui60pjXAhXJbhQKHeffCEwQ8wII2QIwAA) · 320€

##### Combos
- [X3](https://store.dji.com/product/zenmuse-x3-gimbal-camera?from=search-result-v2) 396€ · [247g](https://www.dji.com/matrice100/info)
- [Z3](https://store.dji.com/product/zenmuse-z3?from=menu_products) 780€ [specs](http://www.dji.com/zenmuse-z3/info#specs)
- [X4S](https://store.dji.com/product/zenmuse-x4s?from=search-result-v2) 520€ [only compatible with Inspire2](https://forum.dji.com/thread-73666-1-1.html)

<a id='Wireless'></a>
### Wireless dongle

- [**AC600 802.11ac 2.4GHz/5GHz**](https://www.amazon.co.uk/dp/B06XZ1DBPZ/ref=psdc_949408031_t5_B01EKE0OVQ?th=1) External antena · 18g · 13€

<a id='Platform'></a>
### Platform
[Calculations](https://docs.google.com/spreadsheets/d/1UFneS__9-L0viD8dhHrdOTUGh9j_4SgYT0UNm2cHwcY/edit?usp=sharing)

- **Flaming wheel 550** [Frame](http://www.dji.com/flame-wheel-arf/spec) [E600](http://www.dji.com/e600/spec) [E300](http://www.dji.com/e300/spec) [Extended arms](http://www.multicopterwarehouse.com/index.php?route=product/product&product_id=410) [Landing kit](https://tienda.stockrc.com/epages/eb0140.sf/es_ES/?ObjectPath=/Shops/eb0140/Products/BH-trenkit) [Frame + E300 280€](https://tienda.stockrc.com/epages/eb0140.sf/es_ES/?ObjectPath=/Shops/eb0140/Products/550k) 
- [**Tarot 680**](http://www.tarot-rc.com/index.php?main_page=product_info&products_id=1074) [helipal specs](http://www.helipal.com/tarot-fy680-pro-hexacopter-frame-set.html) · [E800 dji](http://www.dji.com/e800/info#specs)  · [Tarot 4006/620KV motor](http://www.tarot-rc.com/index.php?main_page=product_info&products_id=1076) [tarot 12'' propeller](http://www.tarot-rc.com/index.php?main_page=product_info&products_id=1015) [tarot paired hobbyking esc 12€](https://hobbyking.com/en_us/hobbywing-x-rotor-esc-40amp-opto.html) [Frame 99€](https://www.banggood.com/es/Tarot-680-Pro-TL68P00-6-Axis-Carbon-Fiber-Folding-Frame-p-913674.html?gmcCountry=ES&currency=EUR&createTmp=1&cur_warehouse=CN&utm_source=googleshopping&utm_medium=cpc_ods&utm_content=heath&utm_campaign=pla-multi-es-pc&gclid=Cj0KCQjwvuDPBRDnARIsAGhuAmZmVVLIc4ZZRLExrTAgQyMPwirApE-tmxMXw2KfjIcAZ6Z7eYJgrWYaAtJpEALw_wcB) [E800 309€](https://rc-innovations.es/pack-motorizacion-e800-quad-dji-innovations?language=es&currency=EUR&gclid=Cj0KCQjwvuDPBRDnARIsAGhuAma6pqYH56ilKTT5V0EmXUKjjrCkj-cWtfoU4_oCzaxWyL7G2583lHsaAh6WEALw_wcB) [Tarot 4006/620KV 22€](https://www.banggood.com/es/Tarot-4006620KV-Multi-axis-Brushless-Motor-TL68P02-p-914875.html?gmcCountry=ES&currency=EUR&createTmp=1&cur_warehouse=CN&utm_source=googleshopping&utm_medium=cpc_ods&utm_content=heath&utm_campaign=pla-multi-es-pc&gclid=Cj0KCQjwvuDPBRDnARIsAGhuAmbIjX7H1d905d-OoL5EAPJX3vlF7j71MwrvMJdcRNoKZ6g77Jhu92MaAnHoEALw_wcB) [x2 12'' tarot propeller 13 dollars](https://www.hobby-wing.com/tarot-1355-propeller-tl2829.html)
- [**F450**](https://store.dji.com/product/f450-naza-m-lite-gps-landing-gear) (from EUROC) [E305](https://www.dji.com/e305/spec)
- [**4000mah 6s 30c lipo**](https://hobbyking.com/en_us/turnigy-battery-4000mah-6s-30c-lipo-pack-xt-60.html)
- [**6000mah 6s 25-50c lipo**](https://hobbyking.com/en_us/turnigy-nano-tech-6000mah-6s-25-50c-lipo-pack.html)
- [**S900**](http://www.dji.com/spreading-wings-s900/info#specs) not very adequate for testbed
- S1000 impossible to fly in testbed
- MBZIR has [4006 KV380 motors](http://store-en.tmotor.com/goods.php?id=440) check for 50% with 15'5 propellers on Thrust for weight per motor

- Neo platform
    - Batteries don't charge well any more and are deprecated
    - The flight control is not realiable at all. 
    - The list goes on according to Fran

- F450 frame + Px4 + a type of Kinect camera

    The flight controller would not have the appropriate gains according to the flight characteristics and weigth distribution.

- Light drone

    - 2 [**SanCloud BeagleBone Enhanced**](http://beagleboard.org/bone)
    - Network 
    - [**Hokuyo UST-10LX**](https://acroname.com/blog/hokuyo-laser-range-finder-comparison)
    - Trying to keep the camera, gimbal, autopilot and frame as off the shelf combo


- Qualcomm
[Dragon Drone Development Kit](https://worldsway.com/product/dragon-drone-development-kit/) requires a lot of assemblage. It is basically just compatible parts.

- Px4 based platform
**[Intel® Aero Platform](https://www.intel.co.uk/content/www/uk/en/drones/falcon-8-drone-brief.html)** Payload 0.8Kg

###### DJI

*Mobile SJK* for selfies and facebook

*Onboard SDK* in very new. 2.0 is worthless. 3.0 is better. It in proprietary. Nothing to do with ROS. Poorly documented.
Important contribution from the ETH Autonomous Lab: [dji_onboard_sdk_ros](https://github.com/ethz-asl/dji_onboard_sdk_ros/graphs/code-frequency) Have sent mail to two of the developers to know about support and compatability with Matrice 600. Both Norman Li and DJI Dev Team say Matrice 600 is supported in their reply.

Have their own *simulator* on works with pure DJI SDK. Official developers have no idea what Gazebo even is. [Third party bridge is available for Matrice 100](https://github.com/caochao39/hku_m100_gazebo). Usefull to try out the SDK. Windows & Mac only => bad compatibility with ROS?



- **DJI Matrice 200**
    [Specs](http://www.dji.com/matrice-200-series/info#specs)
    [Software versions compatibility](http://developer.dji.com/onboard-sdk/documentation/development-workflow/environment-setup.html)
    - **No Gazebo** 
    - **Ubuntu** 16.04
    - **Ros version** kinetic
    - **Ros build tool** catkin build
    - **Payload** Max Payload (2 TB50 + full payload), ~2.34kg, 13min  | (2 TB55 + full payload) ~1.61kg, 24min 
    - **Autopilot** 
    - **Onboard CPU/GPU** onboard-sdk


- **DJI Matrice 600**    

    This seems the fastest reliable option so far. Have written to both dev and sales support to get more information (19/10/2017). Their reply was only read the docs and that Matrice 600 is supported.

- **DJI Matrice 600 + LIDAR**

    [LiDAR Mapping](https://developer.dji.com/onboard-sdk/documentation/modules/lidarmapping/lidar-mapping.html)
    [LiDAR-based Collision Avoidance](https://developer.dji.com/onboard-sdk/documentation/modules/collision-avoidance/collision-avoidance.html)
    VLP-16 + Intel NUC i5 (256G SSD and 8G RAM) + !no guidance + !It is with pre-defined waypoint set!

    Collision avoidance:
    - Velodyne driver for ROS
    - DJI Onboard SDK
    - DJI Onboard SDK ROS package
    - DJI collision avoidance package (binary)

- **DJI Matrice 100 + Guidance**
    [Specs](http://www.dji.com/matrice100/info#specs)
    [Software versions compatibility](http://developer.dji.com/onboard-sdk/documentation/development-workflow/environment-setup.html)
    [guidance specs](http://www.dji.com/guidance/info#specs)
    [Gazebo ROS package ETH](https://github.com/ymc-as/m100_gazebo)

    - **No Gazebo** 
    - **Ubuntu** 16.04
    - **Ros version** kinetic
    - **Ros build tool** catkin build
    - **Payload** Max Payload (1 TB48D + camera), ~247g, 23min  | (2 TB48D) ~247g, 23min 
    - **Autopilot** A3 / N3
    - **Onboard CPU/GPU** None
    - **Guidance weight** 64+43+12 = 119g
    - **VLP-16** No gimbal support. In-house integration


- [**DJI Matrice 100 + LIDAR**](https://developer.dji.com/onboard-sdk/documentation/sensor-integration-guides/velodyne/readme.html)

- **DJI Matrice 100 + Px4**
[Setup](https://docs.px4.io/en/frames_multicopter/matrice100.html)


<a id='Summary'></a>
### Summary 2017/11/01

Since the thesis is targeted at large structures, to answer its question "Is it possible to map structures the size of oil rigs with this approach?" the minimum requirement is that a scale factor can be identified between the use case and the tests. The hard constraints mostly come from aspects that cannot be directly scaled and thus require more on the experimental setting.
  Factors that need to be scaled:
  - Structure / volume size.
  - Inner map resolution
  - RAM
  - Safety distance
  - Positioning resolution

**RAM and board**

Previously I was having troubles with the pair inner map resolution and RAM. But I have now found a decoupling from output 3D reconstruction resolution that works, according to my projections. An UPboard processor should be enough for everything.

**Indoors/Outdoors testing**

  Another issue is the coupling of scaling of the size of the structure and the safety distance. I was collecting information from the pilots at CATEC, and from them 3m emerged as the minimum reasonable safety distance in an outdoor inspection environment. Scaling this to the testbed it would become 0.1m, which is not reasonable at all. From this, I assume that the testbed trials must be followed by outdoor trials. After talking to Antidio I realized that this distance in automatic mode is even smaller.

**Point cloud sensor**

  From this discrepancy emerged the need to have both indoors and outdoors tests. Which totally discards the possibility to have any camera as a point cloud sensor in both settings. In the interest of time, I am looking for a solution with minimal changes between indoor and outdoor. An interesting laser sensor is the Hokuyo UST-10LX, it is light and less expensive than a Velodyne. The restriction to 2D capabilities can be overcome mounting the sensor almost vertically and spinning the whole platform for the 360⁰ swipe. This would actually take advantage of the characteristic tilt involved in quadrotor motion.

**Navigation and positioning**

  As for navigation and positioning, the Pixhawk (first version) seems to be the best choice for maximum ROS compatibility along with Vicon integration (Pixhawk 2 has issues) and a Here+ RTK for positioning outdoors.

**Pictures for 3D reconstruction**

  As the end goal is to prove coverage with 3D reconstruction as a use case, the final testing platform should be able to lift both a gimbal and a camera. Escaping DJI lock was not easy but I found a tarot gimbal that should do the trick paired with a GoPro. The Geo reference on the pictures should be enough to prove coverage even if not to generate a usable map.

** Platform**

  The Tarot 680Pro paired with either the E800 dji propulsion pack or with a combination of tarot 12'' propellers, tarot TL6802 motors and a Hobbyking ESC can lift the payload with over a 1Kg of margin.
  From the platforms already at CATEC, MBZIR seem also viable options.

<a id='FinalSetup'></a>
### Final Setup - changes

**Navigation and positioning**

Pixhawk and Tersus

**3D reconstruction**

Going with just point cloud and using Google Cartographer + Potree to build a 3D map.

** Platform **

  MBZIR

<a id='software'></a>
# Software Criteria

- **Vicon interface** is ROS_VRPN_CLIENT
- **Gazebo out of the box** Right now there is no active project using Gazebo at all. José Joaquim says he made his own interface but that it is for indigo. Same problems as with GRVC probably turn up. This is why it really should be out of the box.
- **Ros version**
- **Payload** Point cloud sensor + camera for later mapping
- **Autopilot**
- **Onboard CPU/GPU** 
- **Docker**

<a id='3dmap'></a>
# 3D Map pipeline

1. Laser (Hokuyo) - out PointCloud2 messages into ROS
1. [Google Cartographer](https://www.google.es/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&cad=rja&uact=8&ved=0ahUKEwjl7oyMr9TXAhWJJ8AKHRvzAUEQFggoMAA&url=https%3A%2F%2Fgithub.com%2Fgooglecartographer&usg=AOvVaw1oVz--oO-04L2GYZsAysrG) - out .ply [addition for export .ply](https://github.com/googlecartographer/cartographer/commit/9031f0533aa34b9a0f6a9bf5df6972a204c88cca) [thread about .ply export](https://groups.google.com/forum/#!topic/google-cartographer/14Q3uG5bJYk)
1. [Potree](http://potree.org/) - out some image file (ate the very least print screen)

<a id='experiment'></a>
# Simulation

### Try with large structures and LIDAR

<a id='experiment'></a>
# Experiment

### Steps
1. Set volume. No prior knowledge
2. Just press go
3. Feed data into ~~pix4d~~ - 3d map pipeline
4. Show map

### What to measure
- Coverage
- Flight time needed in autonomous mode
- Flight time and planning time needed in 3d mapping pipepline
- Resolution